<a href="https://colab.research.google.com/github/DarleneJD/ACOPF/blob/main/13barrasOPF_Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acoplamento MT/BT

In [118]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [119]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [120]:
# if not (shutil.which("glpk") or os.path.isfile("glpk")):
#     if "google.colab" in sys.modules:
#         !apt-get install -y -qq glpk-utils
#     else:
#         try:
#             !conda install -c conda-forge glpk
#         except:
#             pass

In [121]:
!git clone https://github.com/DarleneJD/ACOPF.git

fatal: destination path 'ACOPF' already exists and is not an empty directory.


In [122]:
import math
from pathlib import Path
import pandas as pd
import pyomo.environ as pyo
from collections import defaultdict
# Solver (HiGHS)
from pyomo.environ import *
import pandas as pd
import numpy as np


In [123]:
!git clone https://github.com/DarleneJD/ACOPF.git


fatal: destination path 'ACOPF' already exists and is not an empty directory.


In [124]:
base_path = "/content/ACOPF"


In [125]:
branches_path = f"{base_path}/branches_1.xlsx"
buses_path    = f"{base_path}/buses_1.xlsx"

df_branches = pd.read_excel(branches_path, sheet_name="MT")
df_trafos   = pd.read_excel(branches_path, sheet_name="Trafos")
df_buses    = pd.read_excel(buses_path,    sheet_name="MT")


In [126]:
def normalize_phases(value):
    """
    Converte entradas como:
    1
    1.0
    '1,2,3'
    '1.0,2.0,3.0'
    '1.2.3'
    em lista de inteiros [1], [1,2,3]
    """
    if pd.isna(value):
        return []

    s = str(value).strip()

    # unifica separadores
    s = s.replace(".", ",")
    parts = [p.strip() for p in s.split(",") if p.strip() != ""]

    phases = []
    for p in parts:
        try:
            ph = int(float(p))
            if ph in [1, 2, 3]:
                phases.append(ph)
        except ValueError:
            pass

    return sorted(set(phases))

# Branches (MT)
df_branches["phase_list"] = df_branches["phase"].apply(normalize_phases)

# Trafos
df_trafos["phase_list"] = df_trafos["phases"].apply(normalize_phases)


In [127]:
# Remover SourceBus
df_buses = df_buses[~df_buses["name"].astype(str).str.contains("Source", case=False)]

# Mapeamento name -> N (id numérico)
bus_id_map = dict(zip(df_buses["name"], df_buses["N"]))

# Lista final de barras MT
buses_mt = df_buses["N"].astype(int).unique().tolist()


In [128]:
# Converter barras l,k para identificador numérico
df_branches["from_bus"] = df_branches["l"].map(bus_id_map)
df_branches["to_bus"]   = df_branches["k"].map(bus_id_map)

# Remover ramos mal mapeados
df_branches = df_branches.dropna(subset=["from_bus", "to_bus"])

df_branches["from_bus"] = df_branches["from_bus"].astype(int)
df_branches["to_bus"]   = df_branches["to_bus"].astype(int)
df_branches["m"]        = df_branches["m"].astype(int)

# Conjunto de ramos MT
branches_mt = df_branches["m"].unique().tolist()


In [129]:
PHASES = [1, 2, 3]

branch_phase = []

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))  # garante inteiro

    raw = row["phase"]

    if pd.isna(raw):
        continue

    s = str(raw).strip()

    # Normalização:
    # troca qualquer separador por vírgula
    s = s.replace(".", ",")
    parts = [p.strip() for p in s.split(",") if p.strip() != ""]

    for p in parts:
        # extrai apenas números válidos
        try:
            ph = int(float(p))
        except ValueError:
            continue

        if ph in PHASES:
            branch_phase.append((m, ph))

branch_phase = sorted(set(branch_phase))


In [130]:
R = {}
X = {}
Imax = {}
from_bus = {}
to_bus = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))  # robusto se vier 1.0

    from_bus[m] = int(row["from_bus"])
    to_bus[m]   = int(row["to_bus"])
    Imax[m]     = float(row["Imax"])

    # USA A COLUNA NORMALIZADA
    for ph in row["phase_list"]:
        R[(m, ph)] = float(row["R"])
        X[(m, ph)] = float(row["X"])



Length = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))
    Length[m] = float(row["Length_m"])

Vbase = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))
    fb = from_bus[m]
    Vbase[m] = float(df_buses.loc[df_buses["N"] == fb, "v_nom_kv"].iloc[0])
Smax = {}

for m in branches_mt:
    Smax[m] = Vbase[m] * Imax[m]  # kVA por fase


In [131]:
# Padronizar mv_bus
df_trafos["mv_bus_num"] = df_trafos["mv_bus"].map(bus_id_map)
df_trafos = df_trafos.dropna(subset=["mv_bus_num"])
df_trafos["mv_bus_num"] = df_trafos["mv_bus_num"].astype(int)

# Placeholder de potência vinda da BT
P_TR = {}
Q_TR = {}

for _, row in df_trafos.iterrows():
    mvb = int(row["mv_bus_num"])

    # USA DIRETAMENTE A LISTA NORMALIZADA
    for ph in row["phase_list"]:
        P_TR[(mvb, ph)] = 0.0
        Q_TR[(mvb, ph)] = 0.0


In [132]:
model = ConcreteModel()

# Conjuntos
model.BUS = Set(initialize=buses_mt)

bus_phase = set()

for (br, ph) in branch_phase:
    bus_phase.add((from_bus[br], ph))
    bus_phase.add((to_bus[br], ph))

model.BUS_PH = Set(dimen=2, initialize=bus_phase)


model.PH  = Set(initialize=PHASES)
model.BR  = Set(initialize=branches_mt)
model.BR_PH = Set(initialize=branch_phase, dimen=2)

# Parâmetros
model.R = Param(model.BR_PH, initialize=R)
model.X = Param(model.BR_PH, initialize=X)

model.from_bus = Param(model.BR, initialize=from_bus)
model.to_bus   = Param(model.BR, initialize=to_bus)

model.P_TR = Param(model.BUS, model.PH, initialize=lambda m,b,ph: P_TR.get((b,ph),0))
model.Q_TR = Param(model.BUS, model.PH, initialize=lambda m,b,ph: Q_TR.get((b,ph),0))

# Variáveis
model.P = Var(model.BR_PH)
model.Q = Var(model.BR_PH)
model.v = Var(model.BUS, model.PH, bounds=(0.95**2, 1.05**2))

model.Ppos = Var(model.BR_PH, domain=NonNegativeReals)
model.Pneg = Var(model.BR_PH, domain=NonNegativeReals)
model.Qpos = Var(model.BR_PH, domain=NonNegativeReals)
model.Qneg = Var(model.BR_PH, domain=NonNegativeReals)
model.Ploss = Var(model.BR_PH, domain=NonNegativeReals)
model.Ploss = Var(model.BR_PH, domain=NonNegativeReals)



This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [133]:
# Potência aparente "equivalente" (L1) por trafo e fase
model.Str_pos = Var(model.BUS_PH, domain=NonNegativeReals)
model.Str_neg = Var(model.BUS_PH, domain=NonNegativeReals)
model.Qtr_pos = Var(model.BUS_PH, domain=NonNegativeReals)
model.Qtr_neg = Var(model.BUS_PH, domain=NonNegativeReals)

# Perdas
model.Ploss_tr = Var(model.BUS_PH, domain=NonNegativeReals)
model.Qloss_tr = Var(model.BUS_PH, domain=NonNegativeReals)


In [134]:
def voltage_drop_rule(m, br, ph):
    i = m.from_bus[br]
    j = m.to_bus[br]
    return m.v[j,ph] == m.v[i,ph] - 2*(
        (m.R[br,ph]*Length[br]) * m.P[br,ph] +
        (m.X[br,ph]*Length[br]) * m.Q[br,ph]
    )



In [135]:
def Ploss_rule(m, br, ph):
    return m.Ploss[br,ph] >= (
        m.R[br,ph] *
        (m.Ppos[br,ph] + m.Pneg[br,ph]
         + m.Qpos[br,ph] + m.Qneg[br,ph])
    )

model.PlossDef = Constraint(model.BR_PH, rule=Ploss_rule)


In [136]:
def active_balance_rule(m, bus, ph):

    inflow = sum(
        m.P[br, ph]
        for (br, p) in m.BR_PH
        if p == ph and m.to_bus[br] == bus
    )

    outflow = sum(
        m.P[br, ph]
        for (br, p) in m.BR_PH
        if p == ph and m.from_bus[br] == bus
    )

    return inflow - outflow == m.P_TR[bus, ph]
def reactive_balance_rule(m, bus, ph):

    inflow = sum(
        m.Q[br, ph]
        for (br, p) in m.BR_PH
        if p == ph and m.to_bus[br] == bus
    )

    outflow = sum(
        m.Q[br, ph]
        for (br, p) in m.BR_PH
        if p == ph and m.from_bus[br] == bus
    )

    return inflow - outflow == m.Q_TR[bus, ph]
model.PBalance = Constraint(model.BUS_PH, rule=active_balance_rule)
model.QBalance = Constraint(model.BUS_PH, rule=reactive_balance_rule)


In [137]:
def P_split_rule(m, br, ph):
    return m.P[br,ph] == m.Ppos[br,ph] - m.Pneg[br,ph]

def Q_split_rule(m, br, ph):
    return m.Q[br,ph] == m.Qpos[br,ph] - m.Qneg[br,ph]

model.Psplit = Constraint(model.BR_PH, rule=P_split_rule)
model.Qsplit = Constraint(model.BR_PH, rule=Q_split_rule)


In [138]:
Vmin = 0.95  # pu

def current_limit_rule(m, br, ph):
    return (
        m.Ppos[br,ph] + m.Pneg[br,ph]
        + m.Qpos[br,ph] + m.Qneg[br,ph]
        <= Smax[br] * Vmin
    )

model.CurrentLimit = Constraint(model.BR_PH, rule=current_limit_rule)



In [139]:
slack_bus = int(df_buses[df_buses["tb"] == 3]["N"].iloc[0])

for ph in PHASES:
    model.v[slack_bus, ph].fix(1.0)


In [140]:
model.obj = Objective(expr=0.0)


In [141]:
solver = SolverFactory("highs")
result = solver.solve(model, tee=True)

print(result.solver.termination_condition)


Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 186 rows; 203 cols; 551 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+03]
  Cost    [0e+00, 0e+00]
  Bound   [0e+00, 0e+00]
  RHS     [8e+02, 3e+03]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-186); columns 0(-203); nonzeros 0(-551) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Objective value     :  0.0000000000e+00
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
optimal
